In [1]:
import pandas as pd
import numpy as np

## Get Data from Yahoo

In [2]:
from findatapy.market import Market, MarketDataRequest, MarketDataGenerator
from findatapy.util import DataConstants, LoggerManager

In [46]:
def load_decade(start_date = "decade", tickers = '^GSPC'):
    logger = LoggerManager.getLogger(__name__)

    market = Market(market_data_generator=MarketDataGenerator())

    DataConstants.market_thread_technique = 'thread'

    # load S&P 500 ETF ticker via wikipedia
    # snp = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    # tickers = snp[0]['Symbol'].to_list()
    
    # download equities data from Yahoo
    md_request = MarketDataRequest(
        start_date=start_date,
        data_source='yahoo',  # use Bloomberg as data source
        tickers=tickers,  # ticker (findatapy)
        fields=['close', 'open', 'high', 'low', 'volume'],  # which fields to download
        vendor_tickers=tickers,  # ticker (Yahoo)
        vendor_fields=['Close', 'Open', 'High', 'Low', 'Volume'])  # which Bloomberg fields to download)


    logger.info("Loading data with threading")

    df = market.fetch_market(md_request)

    logger.info("Loading data with multiprocessing")

    DataConstants.market_thread_technique = 'multiprocessing'

    df = market.fetch_market(md_request)

    logger.info("Loaded data with multiprocessing")

    return df

In [49]:
from findatapy.util import SwimPool; SwimPool()

spy_df = load_decade(start_date = '01 Jan 1990')

2021-02-09 21:02:22,527 - __main__ - INFO - Loading data with threading
2021-02-09 21:02:22,532 - findatapy.market.datavendorweb - INFO - Request Yahoo data


C:\ProgramData\Anaconda3\lib\site-packages\findatapy\market\marketdatagenerator.py:379: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.
  if expiry_date < start_date:


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
2021-02-09 21:02:23,851 - findatapy.market.datavendorweb - INFO - Completed request from Yahoo.
2021-02-09 21:02:25,896 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_739__abstract_curve_key-None__base_depos_currencies-EUR_GBP_AUD_NZD_USD_CAD_CHF_NOK_SEK_JPY__base_depos_tenor-ON_TN_SN_1W_2W_3W_1M_2M_3M_4M_6M_9M_1Y_2Y_3Y_5Y__category-None__category_key-backtest_default-cat_yahoo_daily_NYC__cut-NYC__data_source-yahoo__environment-backtest__expiry_date-NaT__fields-close_open_high_low_volume__finish_date-2021-02-10 04:44:11_402634__freq-daily__freq_mult-1__fx_forwards_tenor-ON_TN_SN_1W_2W_3W_1M_2M_3M_4M_6M_9M_1Y_2Y_3Y_5Y__fx_vol_part-V_25R_10R_25B_10B__fx_vol_tenor-ON_1W_2W_3W_1M_2M_3M_4M_6M_9M_1Y_2Y_3Y_5Y__gran_freq-None__push_to_cache-True__resample-None__resample_how-last__start_date-1990-01-01 00:00:00__tickers-^GSPC__trade_s

C:\ProgramData\Anaconda3\lib\site-packages\findatapy\market\marketdatagenerator.py:379: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.
  if expiry_date < start_date:


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
2021-02-09 21:02:26,933 - findatapy.market.datavendorweb - INFO - Completed request from Yahoo.
2021-02-09 21:02:28,974 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_739__abstract_curve_key-None__base_depos_currencies-EUR_GBP_AUD_NZD_USD_CAD_CHF_NOK_SEK_JPY__base_depos_tenor-ON_TN_SN_1W_2W_3W_1M_2M_3M_4M_6M_9M_1Y_2Y_3Y_5Y__category-None__category_key-backtest_default-cat_yahoo_daily_NYC__cut-NYC__data_source-yahoo__environment-backtest__expiry_date-NaT__fields-close_open_high_low_volume__finish_date-2021-02-10 04:44:11_402634__freq-daily__freq_mult-1__fx_forwards_tenor-ON_TN_SN_1W_2W_3W_1M_2M_3M_4M_6M_9M_1Y_2Y_3Y_5Y__fx_vol_part-V_25R_10R_25B_10B__fx_vol_tenor-ON_1W_2W_3W_1M_2M_3M_4M_6M_9M_1Y_2Y_3Y_5Y__gran_freq-None__push_to_cache-True__resample-None__resample_how-last__start_date-1990-01-01 00:00:00__tickers-^GSPC__trade_s

In [50]:
spy_df = spy_df.rename(columns={
    "^GSPC.close":"Close", 
    "^GSPC.open":"Open", 
    "^GSPC.high":"High", 
    "^GSPC.low":"Low", 
    "^GSPC.volume":"Volume"})

print("Days:", len(spy_df))

spy_df.tail()

Days: 7838


,Close,Open,High,Low,Volume
Date,,,,,
2021-02-03,3830.169922,3840.270020,3847.510010,3816.679932,4.846900e+09
2021-02-04,3871.739990,3836.659912,3872.419922,3836.659912,4.856670e+09
2021-02-05,3886.830078,3878.300049,3894.560059,3874.929932,4.838580e+09
2021-02-08,3915.590088,3892.590088,3915.770020,3892.590088,4.635030e+09
2021-02-09,3911.229980,3910.489990,3918.350098,3902.639893,2.063161e+09


In [51]:
for col in list(spy_df.columns):
    spy_df[col] = spy_df[col].astype(np.float64)

### load SMA from TA-lib

In [52]:
from talib import abstract
# directly import sma
SMA = abstract.SMA

### Backtest with SMA

In [57]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the a MA lags as *class variables*
    # for later optimization
    n = 25
    
    def init(self):
        # Precompute the moving averages
        self.sma = self.I(SMA, self.data.Close, self.n)
    
    def next(self):
        # If closing price crosses above sma, close any existing
        # short trades, and buy the asset
        if crossover(self.data.Close, self.sma):
            self.position.close()
            self.buy()

        # Else, if closing price crosses below sma, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma, self.data.Close):
            self.position.close()
            self.sell()

In [58]:
from backtesting import Backtest

bt = Backtest(spy_df, SmaCross, cash=10_000, commission=.0)
stats = bt.run()
stats

Start                     1990-01-02 00:00:00
End                       2021-02-09 00:00:00
Duration                  11361 days 00:00:00
Exposure Time [%]                   99.578974
Equity Final [$]                   5151.03006
Equity Peak [$]                  13038.097717
Return [%]                         -48.489699
Buy & Hold Return [%]              987.389128
Return (Ann.) [%]                   -2.110279
Volatility (Ann.) [%]               14.581021
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -71.208845
Avg. Drawdown [%]                   -5.274172
Max. Drawdown Duration    10934 days 00:00:00
Avg. Drawdown Duration      513 days 00:00:00
# Trades                                  825
Win Rate [%]                        24.484848
Best Trade [%]                      21.507043
Worst Trade [%]                    -10.824752
Avg. Trade [%]                    

## With Stochastic Indicator

https://www.investopedia.com/articles/technical/073001.asp

In [26]:
# directly import stochastic
STOCH = abstract.STOCH
# uses high, low, close (default) to compute K line and D line
slowk, slowd = STOCH(high=spy_df['high'], low=spy_df['low'], close=spy_df['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

KeyError: 'high'